In [81]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime

In [83]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [84]:
cwd = os.getcwd()
print(cwd)

/cluster/home/skyriakos/chemprop_run/git/notebooks


In [85]:
os.chdir('..')
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.run_workflow import *
from deepadr.chemfeatures import *
from deepadr.hyphelper import *
# from deepadr.model_gnn import GCN as testGCN
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN
# from deepadr.model_attn_siamese import *
from ogb.graphproppred import Evaluator
os.chdir(cwd)

In [86]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [87]:
# from tdc.single_pred import Tox
# from tdc.multi_pred import DDI

In [88]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [89]:
# report_available_cuda_devices()

In [90]:
n_gpu = torch.cuda.device_count()
n_gpu

5

In [91]:
device_cpu = get_device(to_gpu=False)
# device_gpu = get_device(True, index=0)

# fdtype = torch.float32

In [92]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

torch: 1.9.1
CUDA: 11.1
3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:41:03) 
[GCC 9.4.0]


In [93]:
# print(torch.cuda.memory_summary(device=device_gpu))

### Preparing dataset 

In [94]:
score = 'zip_thresh'
score_val = 1

In [95]:
# TDC Tox
# DSdataset_name = 'SynergxZloewe' #'OncoPolyPharmacology' #'DrugComb'
DSdataset_name = f'DrugComb_{score}_{score_val}' #'DrugComb'


#fname_suffix = ds_config["fname_suffix"]
similarity_types = ['chem']
kernel_option = 'sqeuclidean'
data_fname = 'data_v1'
# interact_matfname = ds_config["interact_matfname"]
# exp_iden = 'simtypeall'2022-03-29_11-04-21
# ddi_interaction_labels_pth = ds_config["ddi_interaction_labels_pth"]

# up_dir, processed_dir, DSdataset_name, data_fname

In [96]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir)

path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
/cluster/home/skyriakos/chemprop_run/git/data/processed/DrugComb_zip_thresh_1/data_v1


In [97]:
# %%time

# # Make sure to first run the "data_generation" notebook first

# dataset = MoleculeDataset(root=targetdata_dir)

In [98]:
# dataset.data.y

In [99]:
# fold_partitions = get_stratified_partitions(dataset.data.y,
#                                             num_folds=5,
#                                             valid_set_portion=0.1,
#                                             random_state=42)

In [100]:
# print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
# print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
# print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))


### Hyper Parameters

In [101]:
time_stamp = "2022-03-31_15-28-25"

import glob
exp_dirs = glob.glob(targetdata_dir_exp+"/fold_*_"+time_stamp)
len(exp_dirs)

5

In [102]:
folds = []

for edir in exp_dirs:
    fold = edir.split("/")[-1].split('_')[1]
    df_curves = pd.read_csv(edir + "/curves.csv")
#     le = df_curves.tail(1)
#     folds.append([fold, le.test_aupr.values[0], le.test_auc.values[0]])
    df_curves["fold"] = [fold] * len(df_curves)
    folds.append(df_curves)

In [103]:
df_folds = pd.concat(folds)
df_folds

,epoch,train_aupr,valid_aupr,test_aupr,train_auc,valid_auc,test_auc,fold
0,0,0.708357,0.705700,0.703337,0.702999,0.699982,0.697960,0
1,1,0.733682,0.730308,0.728842,0.733003,0.731079,0.727612,0
2,2,0.758322,0.756716,0.751152,0.751650,0.749879,0.744667,0
3,3,0.776156,0.768535,0.762334,0.765823,0.761271,0.754948,0
4,4,0.788562,0.781338,0.773732,0.780547,0.775384,0.769624,0
...,...,...,...,...,...,...,...,...
95,95,0.996991,0.939853,0.938653,0.996631,0.944474,0.942188,4
96,96,0.998401,0.943220,0.944066,0.998231,0.946349,0.945889,4
97,97,0.998996,0.943141,0.944196,0.998873,0.947064,0.947025,4
98,98,0.999415,0.944654,0.945744,0.999344,0.949566,0.948287,4


In [104]:
df_folds_epoch = df_folds.groupby("epoch")[["test_aupr", "test_auc"]].mean()
df_folds_epoch["Fscore"] = df_folds_epoch.apply(lambda x: (2*x[0]*x[1])/(x[0]+x[1]), axis=1) # harmonic mean of AUC, AUPR
df_folds_epoch

,test_aupr,test_auc,Fscore
epoch,,,
0,0.710442,0.699992,0.705178
1,0.736180,0.729426,0.732788
2,0.756528,0.750920,0.753714
3,0.772153,0.765679,0.768902
4,0.784054,0.780071,0.782058
...,...,...,...
95,0.940822,0.943446,0.942132
96,0.943504,0.946132,0.944816
97,0.945179,0.947795,0.946485


In [105]:
best_epoch = df_folds_epoch.Fscore.idxmax()
df_folds_epoch.loc[best_epoch]

test_aupr    0.948165
test_auc     0.950594
Fscore       0.949378
Name: 99, dtype: float64

In [106]:
best_epoch

99

### Attention Scores

In [107]:
gene_gex = pd.read_csv('../data/synergxdb/RMA/gene_gex.tsv', sep='\t')
dict_gene_gex = dict(zip(gene_gex.GEX, gene_gex.GENE_SYMBOLS))
gex = list(gene_gex.GEX)

In [108]:
l_attn = []

for edir in exp_dirs:
    attn_scores = pd.read_csv(edir + "/fattn_w_scores_e_ids_test.csv")
    l_attn.append(attn_scores)

In [109]:
df_attn = pd.concat(l_attn, axis=0).drop(columns="Unnamed: 0").astype({"id":int}).set_index("id")
df_attn

,gex0,gex1,gex2,gex3,gex4,gex5,gex6,gex7,gex8,gex9,...,gex898,gex899,gex900,gex901,gex902,gex903,gex904,gex905,gex906,gex907
id,,,,,,,,,,,,,,,,,,,,,
15,0.000820,0.000913,0.002205,0.000491,0.000425,0.000274,0.001143,0.001592,0.000562,0.000205,...,0.000442,0.000193,0.000221,0.000810,0.000239,0.000205,0.000389,0.000529,0.001570,0.001493
26,0.000820,0.000913,0.002205,0.000491,0.000425,0.000274,0.001143,0.001592,0.000562,0.000205,...,0.000442,0.000193,0.000221,0.000810,0.000239,0.000205,0.000389,0.000529,0.001570,0.001493
38,0.000820,0.000913,0.002205,0.000491,0.000425,0.000274,0.001143,0.001592,0.000562,0.000205,...,0.000442,0.000193,0.000221,0.000810,0.000239,0.000205,0.000389,0.000529,0.001570,0.001493
39,0.000690,0.001786,0.001836,0.000301,0.000955,0.002915,0.000560,0.000656,0.001013,0.000212,...,0.000415,0.000258,0.001914,0.001614,0.000346,0.000483,0.000937,0.000292,0.002451,0.002741
42,0.000690,0.001786,0.001836,0.000301,0.000955,0.002915,0.000560,0.000656,0.001013,0.000212,...,0.000415,0.000258,0.001914,0.001614,0.000346,0.000483,0.000937,0.000292,0.002451,0.002741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88920,0.000773,0.000353,0.002021,0.000510,0.000642,0.000651,0.001100,0.001709,0.001059,0.000553,...,0.002099,0.000453,0.001806,0.000333,0.000541,0.000410,0.000691,0.000383,0.001710,0.001400
88926,0.000773,0.000353,0.002021,0.000510,0.000642,0.000651,0.001100,0.001709,0.001059,0.000553,...,0.002099,0.000453,0.001806,0.000333,0.000541,0.000410,0.000691,0.000383,0.001710,0.001400
88930,0.000773,0.000353,0.002021,0.000510,0.000642,0.000651,0.001100,0.001709,0.001059,0.000553,...,0.002099,0.000453,0.001806,0.000333,0.000541,0.000410,0.000691,0.000383,0.001710,0.001400


In [110]:
sns.set(rc={'figure.figsize':(30,30)})


# sns.heatmap(df_attn)

In [111]:
# sns.clustermap(df_attn)

In [112]:
1 / 908.0

0.0011013215859030838

In [113]:
def top_attn_genes(gex, nrhead):
    return gex.mean().sort_values(ascending=False).head(nrhead).rename(index=dict_gene_gex)

In [114]:
top_attn_genes(df_attn[gex], 10)

PTPRC     0.001679
IKZF1     0.001450
COL1A1    0.001379
PAX8      0.001350
BTK       0.001333
PRR15L    0.001274
SPDEF     0.001248
CCNA1     0.001244
CXCR4     0.001240
CORO1A    0.001233
dtype: float64

### Predictions (Best Epoch)

In [115]:
print("best epoch:", best_epoch)

l_pred = []

for edir in exp_dirs:
    predictions = pd.read_csv(edir + f"/predictions/epoch_{best_epoch}_predictions_test.csv")
    l_pred.append(predictions)

best epoch: 99


In [116]:
df_pred = pd.concat(l_pred, axis=0).astype({"id":int}).set_index("id")
df_pred

,true_class,pred_class,prob_score_class0,prob_score_class1
id,,,,
15,1,1,0.060660,0.939340
26,1,1,0.029464,0.970537
38,1,1,0.113637,0.886363
39,1,1,0.052378,0.947622
42,1,1,0.038269,0.961731
...,...,...,...,...
88920,1,0,0.995783,0.004217
88926,0,0,0.897474,0.102526
88930,0,1,0.173412,0.826588


### DATA

In [117]:
data_pairs = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'data_pairs.pkl'))
data_pairs

,Drug1_ID,Drug2_ID,Cell_Line_ID,Cosmic_ID,Drug1,Drug2,Y
0,dacarbazine,717906-29-1,A-673,684052,CN(C)N=NC1=C(NC=N1)C(=O)N,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0
1,CHEMBL3348974,717906-29-1,A-673,684052,CC(C)OC(=O)C(C)NP(=O)(OCC1C(C(C(O1)N2C=CC(=O)N...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1
2,Nesbuvir,717906-29-1,A-673,684052,CNC(=O)C1=C(OC2=CC(=C(C=C21)C3CC3)N(CCO)S(=O)(...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0
3,BMS-599626,717906-29-1,A-673,684052,CC1=C2C(=NC=NN2C=C1NC(=O)OCC3COCCN3)NC4=CC5=C(...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1
4,WZ8040,717906-29-1,A-673,684052,CN1CCN(CC1)C2=CC=C(C=C2)NC3=NC=C(C(=N3)SC4=CC=...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1
...,...,...,...,...,...,...,...
88939,Vorinostat,KU-60019,VM-CUB-1,909780,C1=CC=C(C=C1)NC(=O)CCCCCCC(=O)NO,CC1CN(CC(O1)C)CC(=O)NC2=CC3=C(C=C2)SC4=C(C=CC=...,1
88940,AZD5582,AZD4320,VM-CUB-1,909780,CC(C(=O)NC(C1CCCCC1)C(=O)N2CCCC2C(=O)NC3C(CC4=...,CN(CCC(CSC1=CC=CC=C1)NC2=C(C=C(C=C2)S(=O)(=O)N...,0
88941,957054-30-7,Sapitinib,VM-CUB-1,909780,CS(=O)(=O)N1CCN(CC1)CC2=CC3=C(S2)C(=NC(=N3)C4=...,CNC(=O)CN1CCC(CC1)OC2=C(C=C3C(=C2)C(=NC=N3)NC4...,1
88942,957054-30-7,Selumetinib,VM-CUB-1,909780,CS(=O)(=O)N1CCN(CC1)CC2=CC3=C(S2)C(=NC(=N3)C4=...,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,1


In [118]:
df_all = pd.concat([data_pairs, df_attn, df_pred], axis=1)
df_all

,Drug1_ID,Drug2_ID,Cell_Line_ID,Cosmic_ID,Drug1,Drug2,Y,gex0,gex1,gex2,...,gex902,gex903,gex904,gex905,gex906,gex907,true_class,pred_class,prob_score_class0,prob_score_class1
0,dacarbazine,717906-29-1,A-673,684052,CN(C)N=NC1=C(NC=N1)C(=O)N,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0,0.000800,0.000894,0.002237,...,0.000222,0.000189,0.000368,0.000507,0.001571,0.001491,0,0,0.854182,0.145818
1,CHEMBL3348974,717906-29-1,A-673,684052,CC(C)OC(=O)C(C)NP(=O)(OCC1C(C(C(O1)N2C=CC(=O)N...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1,0.000800,0.000894,0.002237,...,0.000222,0.000189,0.000368,0.000507,0.001571,0.001491,1,1,0.395283,0.604717
2,Nesbuvir,717906-29-1,A-673,684052,CNC(=O)C1=C(OC2=CC(=C(C=C21)C3CC3)N(CCO)S(=O)(...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0,0.000820,0.000912,0.002206,...,0.000238,0.000204,0.000388,0.000528,0.001570,0.001493,0,0,0.635120,0.364880
3,BMS-599626,717906-29-1,A-673,684052,CC1=C2C(=NC=NN2C=C1NC(=O)OCC3COCCN3)NC4=CC5=C(...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1,0.000820,0.000912,0.002206,...,0.000238,0.000204,0.000388,0.000528,0.001570,0.001493,1,1,0.350460,0.649540
4,WZ8040,717906-29-1,A-673,684052,CN1CCN(CC1)C2=CC=C(C=C2)NC3=NC=C(C(=N3)SC4=CC=...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1,0.000951,0.001028,0.001956,...,0.000387,0.000346,0.000552,0.000691,0.001527,0.001472,1,1,0.034023,0.965977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88939,Vorinostat,KU-60019,VM-CUB-1,909780,C1=CC=C(C=C1)NC(=O)CCCCCCC(=O)NO,CC1CN(CC(O1)C)CC(=O)NC2=CC3=C(C=C2)SC4=C(C=CC=...,1,0.000622,0.000212,0.002329,...,0.000381,0.000261,0.000533,0.000237,0.001851,0.001407,1,0,0.922535,0.077465
88940,AZD5582,AZD4320,VM-CUB-1,909780,CC(C(=O)NC(C1CCCCC1)C(=O)N2CCCC2C(=O)NC3C(CC4=...,CN(CCC(CSC1=CC=CC=C1)NC2=C(C=C(C=C2)S(=O)(=O)N...,0,0.000623,0.000212,0.002328,...,0.000382,0.000261,0.000534,0.000238,0.001851,0.001407,0,1,0.026859,0.973141
88941,957054-30-7,Sapitinib,VM-CUB-1,909780,CS(=O)(=O)N1CCN(CC1)CC2=CC3=C(S2)C(=NC(=N3)C4=...,CNC(=O)CN1CCC(CC1)OC2=C(C=C3C(=C2)C(=NC=N3)NC4...,1,0.000623,0.000212,0.002328,...,0.000382,0.000261,0.000534,0.000238,0.001851,0.001407,1,1,0.001204,0.998796
88942,957054-30-7,Selumetinib,VM-CUB-1,909780,CS(=O)(=O)N1CCN(CC1)CC2=CC3=C(S2)C(=NC(=N3)C4=...,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,1,0.000623,0.000212,0.002328,...,0.000382,0.000261,0.000534,0.000238,0.001851,0.001407,1,1,0.005207,0.994793


In [119]:
dir_fold_all = create_directory(os.path.join(targetdata_dir_exp, "all_fold_" + time_stamp))
print(dir_fold_all)

path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
/cluster/home/skyriakos/chemprop_run/git/data/processed/DrugComb_zip_thresh_1/data_v1/experiments/all_fold_2022-03-31_15-28-25


In [120]:
ReaderWriter.dump_data(df_all, os.path.join(dir_fold_all, f'drugcomb_attn_predictions_{score}_{score_val}.pkl'))

In [ ]:
df_all['Cell_Line_ID'].value_counts().head(40)

In [ ]:
sel_cl = "COLO 800"

sel_gex = df_all[df_all['Cell_Line_ID'] == sel_cl][gex]
sel_gex

In [ ]:
sns.heatmap(sel_gex)

In [ ]:
sns.clustermap(sel_gex, col_cluster=False)

In [ ]:
exp = 1 / len(gex)
exp

In [ ]:
df_mean = pd.DataFrame(sel_gex.mean()).T
df_mean.T.describe()#.loc['75%'].values[0]

In [ ]:
df_mean_thresh = df_mean.copy()
thresh = df_mean.T.describe().loc['75%'].values[0]
df_mean_thresh[df_mean_thresh < thresh] = 0
df_mean_thresh

In [ ]:
sns.set(rc={'figure.figsize':(30,1)})

sns.heatmap(df_mean)

In [ ]:
top_attn_genes(sel_gex, 10)

In [ ]:
sel_gex.mean().sort_values(ascending=False).head(10).rename(index=dict_gene_gex)